In [ ]:
import os

from fairseq.data import (
    data_utils,
    Dictionary,
    AppendTokenDataset,
    PrependTokenDataset,
    StripTokenDataset,
    TokenBlockDataset,
)
from .denoised_dataset import BARTDenoisingDataset
from fairseq.data.encoders.utils import get_whole_word_mask
from fairseq.tasks import FairseqTask, register_task


@register_task('mask_s2s')
class DenoisingTaskS2S(FairseqTask):
    """
    Denoising task for applying sequence to sequence denoising.
    """

    @staticmethod
    def add_args(parser):
        """Add task-specific arguments to the parser."""
        parser.add_argument('data', help='path to data directory')
        parser.add_argument('--tokens-per-sample', default=512, type=int,
                            help='max number of total tokens over all segments'
                                 ' per sample for dataset')
        parser.add_argument('--raw-text', default=False, action='store_true',
                            help='load raw text dataset')
        parser.add_argument(
            '--sample-break-mode', default="eos", type=str,
            help='mode for breaking sentence',
        )
        parser.add_argument(
            '--mask', default=0.3, type=float,
            help='fraction of words/subwords that will be masked',
        )
        parser.add_argument(
            '--mask-random', default=0.0, type=float,
            help='instead of using [MASK], use random token this often'
        )
        parser.add_argument(
            '--insert', default=0.0, type=float,
            help='insert this percentage of additional random tokens',
        )
        parser.add_argument(
            '--mask-length', default="word", type=str,
            choices=['subword', 'word', 'span', 'span-poisson'],
            help='mask length to choose'
        )
        parser.add_argument(
            '--replace-length', default=-1, type=int,
            help='when masking N tokens, replace with 0, 1, or N tokens (use -1 for N)'
        )
        parser.add_argument(
            '--tokens-to-keep', default=2, type=int,
            help="Don't mask first <tokens-to-keep> tokens"
        )

        # following 2 arguments are required for the GPT2 BPE encoding
        parser.add_argument(
            '--gpt2_encoder_json', default=None, type=str,
            help='GPT2 encoder path'
        )
        parser.add_argument(
            '--gpt2_vocab_bpe', default=None, type=str,
            help='GPT2 vocab path'
        )

    def __init__(self, args, dictionary):
        super().__init__(args)
        self.dictionary = dictionary
        self.seed = args.seed

        # add mask token
        self.mask_idx = self.dictionary.add_symbol('<mask>')

    @classmethod
    def setup_task(cls, args, **kwargs):
        """Setup the task.
        """
        dictionary = Dictionary.load(os.path.join(args.data, 'dict.txt'))
        print('| dictionary: {} types'.format(len(dictionary)))
        if not hasattr(args, 'shuffle_instance'):
            args.shuffle_instance = False
        return cls(args, dictionary)

    def load_dataset(self, split, epoch=0, combine=False, data_selector=None):
        """Load a given dataset split.

        Args:
            split (str): name of the split (e.g., train, valid, test)
        """

        paths = self.args.data.split(':')
        assert len(paths) > 0
        data_path = paths[epoch % len(paths)]
        split_path = os.path.join(data_path, split)

        dataset = data_utils.load_indexed_dataset(
            split_path,
            self.dictionary,
            self.args.dataset_impl,
            combine=combine,
        )
        if dataset is None:
            raise FileNotFoundError('Dataset not found: {} ({})'.format(split, split_path))

        dataset = StripTokenDataset(dataset, self.dictionary.eos())

        # create continuous blocks of tokens
        dataset = TokenBlockDataset(
                dataset,
                dataset.sizes,
                self.args.tokens_per_sample - 2,  # one less for <s> and one for </s>
                pad=self.dictionary.pad(),
                eos=self.dictionary.eos(),
                break_mode="eos",
                document_sep_len=0
        )

        # prepend beginning-of-sentence token (<s>, equiv. to [CLS] in BERT)
        dataset = PrependTokenDataset(dataset, self.source_dictionary.bos())
        dataset = AppendTokenDataset(dataset, self.source_dictionary.eos())

        mask_whole_words = get_whole_word_mask(self.args, self.source_dictionary) \
            if self.args.mask_length != 'subword' else None

        self.datasets[split] = BARTDenoisingDataset(
            dataset, dataset.sizes, self.dictionary, self.mask_idx,
            mask_whole_words, shuffle=self.args.shuffle_instance,
            seed=self.seed, args=self.args
        )
        print(
            "| Split: {0}, Loaded {1} samples of denoising_dataset".format(
                split,
                len(self.datasets[split]),
            )
        )

    def max_positions(self):
        """Return the max sentence length allowed by the task."""
        #return (self.args.max_source_positions, self.args.max_target_positions)
        return (1024, 1024)

    @property
    def source_dictionary(self):
        """Return the source :class:`~fairseq.data.Dictionary`."""
        return self.dictionary

    @property
    def target_dictionary(self):
        """Return the target :class:`~fairseq.data.Dictionary`."""
        return self.dictionary


In [ ]:
import numpy as np
import torch
import math

from fairseq.data import data_utils, FairseqDataset


def collate(
    samples,
    pad_idx,
    eos_idx,
    vocab,
    left_pad_source=False,
    left_pad_target=False,
    input_feeding=True,
):
    assert input_feeding
    if len(samples) == 0:
        return {}

    def merge(key, left_pad, move_eos_to_beginning=False):
        return data_utils.collate_tokens(
            [s[key] for s in samples],
            pad_idx, eos_idx, left_pad, move_eos_to_beginning,
        )

    id = torch.LongTensor([s['id'] for s in samples])
    src_tokens = merge('source', left_pad=left_pad_source)
    # sort by descending source length
    src_lengths = torch.LongTensor([s['source'].numel() for s in samples])
    src_lengths, sort_order = src_lengths.sort(descending=True)
    id = id.index_select(0, sort_order)
    src_tokens = src_tokens.index_select(0, sort_order)

    prev_output_tokens = None
    target = None
    if samples[0].get('target', None) is not None:
        target = merge('target', left_pad=left_pad_target)
        target = target.index_select(0, sort_order)
        ntokens = sum(len(s['target']) for s in samples)

        if input_feeding:
            # we create a shifted version of targets for feeding the
            # previous output token(s) into the next decoder step
            prev_output_tokens = merge(
                'target',
                left_pad=left_pad_target,
                move_eos_to_beginning=True,
            )
            prev_output_tokens = prev_output_tokens.index_select(0, sort_order)
    else:
        ntokens = sum(len(s['source']) for s in samples)

    batch = {
        'id': id,
        'ntokens': ntokens,
        'net_input': {
            'src_tokens': src_tokens,
            'src_lengths': src_lengths,
        },
        'target': target,
        'nsentences': samples[0]['source'].size(0),
    }
    if prev_output_tokens is not None:
        batch['net_input']['prev_output_tokens'] = prev_output_tokens
    return batch


class BARTDenoisingDataset(FairseqDataset):
    """
    A wrapper around TokenBlockDataset for BART dataset.

    Args:
        dataset (TokenBlockDataset): dataset to wrap
        sizes (List[int]): sentence lengths
        vocab (~fairseq.data.Dictionary): vocabulary
        mask_idx (int): dictionary index used for masked token
        mask_whole_words: only mask whole words. This should be a byte mask
            over vocab indices, indicating whether it is the beginning of a
            word. We will extend any mask to encompass the whole word.
        shuffle (bool, optional): shuffle the elements before batching.
          Default: ``True``
        seed: Seed for random number generator for reproducibility.
        args: argparse arguments.
    """

    def __init__(
        self,
        dataset,
        sizes,
        vocab,
        mask_idx,
        mask_whole_words,
        shuffle,
        seed,
        args
    ):
        self.dataset = dataset

        self.sizes = sizes

        self.vocab = vocab
        self.shuffle = shuffle
        self.seed = seed
        self.mask_idx = mask_idx
        self.mask_whole_word = mask_whole_words
        self.mask_ratio = args.mask
        self.random_ratio = args.mask_random
        self.insert_ratio = args.insert
        self.tokens_to_keep = args.tokens_to_keep

        if args.bpe != 'gpt2':
            self.full_stop_index = self.vocab.index(".")
        else:
            assert args.bpe == 'gpt2'
            self.full_stop_index = self.vocab.index('13')

        self.tab_sep_index = self.vocab.index('\t')
        self.replace_length = args.replace_length
        if not self.replace_length in [-1, 0, 1]:
            raise (f'invalid arg: replace_length={self.replace_length}')
        if not args.mask_length in ['subword', 'word', 'span', 'span-poisson']:
            raise (f'invalid arg: mask-length={args.mask_length}')
        if args.mask_length == 'subword' and not args.replace_length in [0, 1]:
            raise (f'if using subwords, use replace-length=1 or 0')

        self.is_span_mask = (args.mask_length == 'span')
        self.mask_span_distribution = None
        if args.mask_length == 'span-poisson':
            _lambda = args.poisson_lambda

            lambda_to_the_k = 1
            e_to_the_minus_lambda = math.exp(-_lambda)
            k_factorial = 1
            ps = []
            for k in range(0, 128):
                ps.append(e_to_the_minus_lambda * lambda_to_the_k / k_factorial)
                lambda_to_the_k *= _lambda
                k_factorial *= (k + 1)
                if ps[-1] < 0.0000001:
                    break
            ps = torch.FloatTensor(ps)
            self.mask_span_distribution = torch.distributions.Categorical(ps)

        self.epoch = 0
        torch.manual_seed(self.seed)

    def set_epoch(self, epoch, **unused):
        self.epoch = epoch

    def __getitem__(self, index):
        with data_utils.numpy_seed(self.seed, self.epoch, index):
            tokens = self.dataset[index]
            assert tokens[-1] == self.vocab.eos()
            source, target = tokens, tokens.clone()

            if self.mask_ratio > 0:
                if self.is_span_mask:
                    source = self.add_multiple_words_mask(source, self.mask_ratio)
                else:
                    source = self.add_whole_word_mask(source, self.mask_ratio)

        assert (source >= 0).all()
        assert (source[1:-1] >= 1).all()
        assert (source <= len(self.vocab)).all()
        assert source[0] == self.vocab.bos()
        assert source[-1] == self.vocab.eos()
        return {
            'id': index,
            'source': source,
            'target': target,
        }

    def __len__(self):
        return len(self.dataset)

    def word_starts(self, source):
        if self.mask_whole_word is not None:
            is_word_start = self.mask_whole_word.gather(0, source)
        else:
            is_word_start = torch.ones(source.size())
        is_word_start[0] = 0
        is_word_start[-1] = 0

        is_word_start[1] = 0  # exclude the first word. Label word
        # for i in range(1, self.tokens_to_keep+1):
        #     is_word_start[i] = 0

        return is_word_start

    def add_whole_word_mask(self, source, p):
        is_word_start = self.word_starts(source)
        num_to_mask = int(math.ceil(is_word_start.float().sum() * p))
        num_inserts = 0
        if num_to_mask == 0:
            return source

        if self.mask_span_distribution is not None:
            lengths = self.mask_span_distribution.sample(sample_shape=(num_to_mask,))

            # Make sure we have enough to mask
            cum_length = torch.cumsum(lengths, 0)
            while cum_length[-1] < num_to_mask:
                lengths = torch.cat([lengths, self.mask_span_distribution.sample(sample_shape=(num_to_mask,))], dim=0)
                cum_length = torch.cumsum(lengths, 0)

            # Trim to masking budget
            i = 0
            while cum_length[i] < num_to_mask:
                i += 1
            lengths[i] = num_to_mask - (0 if i == 0 else cum_length[i - 1])
            num_to_mask = i + 1
            lengths = lengths[:num_to_mask]

            # Handle 0-length mask (inserts) separately
            lengths = lengths[lengths > 0]
            num_inserts = num_to_mask - lengths.size(0)
            num_to_mask -= num_inserts
            if num_to_mask == 0:
                return self.add_insertion_noise(source, num_inserts / source.size(0))

            assert (lengths > 0).all()
        else:
            lengths = torch.ones((num_to_mask,)).long()

        assert is_word_start[-1] == 0
        word_starts = is_word_start.nonzero()
        indices = word_starts[torch.randperm(word_starts.size(0))[:num_to_mask]].squeeze(1)
        mask_random = torch.FloatTensor(num_to_mask).uniform_() < self.random_ratio

        source_length = source.size(0)
        assert source_length - 1 not in indices
        to_keep = torch.ones(source_length, dtype=torch.bool)
        is_word_start[-1] = 255 # acts as a long length, so spans don't go over the end of doc
        if self.replace_length == 0:
            to_keep[indices] = 0
        else:
            # keep index, but replace it with [MASK]
            source[indices] = self.mask_idx
            source[indices[mask_random]] = torch.randint(1, len(self.vocab), size=(mask_random.sum(),))

        if self.mask_span_distribution is not None:
            assert len(lengths.size()) == 1
            assert lengths.size() == indices.size()
            lengths -= 1
            while indices.size(0) > 0:
                assert lengths.size() == indices.size()
                lengths -= is_word_start[indices + 1].long()
                uncompleted = lengths >= 0
                indices = indices[uncompleted] + 1
                mask_random = mask_random[uncompleted]
                lengths = lengths[uncompleted]
                if self.replace_length != -1:
                    # delete token
                    to_keep[indices] = 0
                else:
                    # keep index, but replace it with [MASK]
                    source[indices] = self.mask_idx
                    source[indices[mask_random]] = torch.randint(1, len(self.vocab), size=(mask_random.sum(),))
        else:
            # A bit faster when all lengths are 1
            while indices.size(0) > 0:
                uncompleted = is_word_start[indices + 1] == 0
                indices = indices[uncompleted] + 1
                mask_random = mask_random[uncompleted]
                if self.replace_length != -1:
                    # delete token
                    to_keep[indices] = 0
                else:
                    # keep index, but replace it with [MASK]
                    source[indices] = self.mask_idx
                    source[indices[mask_random]] = torch.randint(1, len(self.vocab), size=(mask_random.sum(),))

                assert source_length - 1 not in indices

        source = source[to_keep]

        if num_inserts > 0:
            source = self.add_insertion_noise(source, num_inserts / source.size(0))

        return source

    def add_multiple_words_mask(self, source, p):
        is_word_start = self.word_starts(source)
        num_to_mask = int(math.ceil(is_word_start.float().sum() * p))
        if num_to_mask == 0:
            return source

        assert is_word_start[-1] == 0
        word_starts = is_word_start.nonzero()
        start_index = word_starts.size(0)-num_to_mask
        if start_index < 1:
            print(source, is_word_start)
            return source

        mask_word_start_id = np.random.randint(start_index)

        source_length = source.size(0)
        to_keep = torch.ones(source_length, dtype=torch.bool)
        is_word_start[-1] = 255 # acts as a long length, so spans don't go over the end of doc

        # keep first index, but replace it with [MASK], and delete remaining index
        source[word_starts[mask_word_start_id]] = self.mask_idx
        #assert mask_word_start_id+num_to_mask < word_starts.size(0)
        #assert (word_starts[mask_word_start_id].item()+num_to_mask) < source_length
        try:
            for ind in range(word_starts[mask_word_start_id]+1, word_starts[mask_word_start_id+num_to_mask]):
                to_keep[ind] = 0
        except IndexError:
            print("Index error", source, is_word_start)
            pass

        source = source[to_keep]
        return source

    def collater(self, samples):
        """Merge a list of samples to form a mini-batch.
        Args:
            samples (List[dict]): samples to collate
        Returns:
            dict: a mini-batch of data
        """
        return collate(samples, self.vocab.pad(), self.vocab.eos(), self.vocab)

    def num_tokens(self, index):
        """Return the number of tokens in a sample. This value is used to
        enforce ``--max-tokens`` during batching."""
        return self.sizes[index]

    def size(self, index):
        """Return an example's size as a float or tuple. This value is used when
        filtering a dataset with ``--max-positions``."""
        return self.sizes[index]

    def ordered_indices(self):
        """Return an ordered list of indices. Batches will be constructed based
        on this order."""
        if self.shuffle:
            indices = np.random.permutation(len(self))
        else:
            indices = np.arange(len(self))
        return indices[np.argsort(self.sizes[indices], kind='mergesort')]

    def prefetch(self, indices):
        self.src.prefetch(indices)
        self.tgt.prefetch(indices)

    @property
    def supports_prefetch(self):
        return (
            hasattr(self.src, 'supports_prefetch')
            and self.src.supports_prefetch
            and hasattr(self.tgt, 'supports_prefetch')
            and self.tgt.supports_prefetch
        )

In [ ]:
import os

from fairseq.data import (
    data_utils,
    Dictionary,
    AppendTokenDataset,
    PrependTokenDataset,
    StripTokenDataset,
    TokenBlockDataset,
)
from .denoised_dataset import BARTDenoisingDataset
from fairseq.data.encoders.utils import get_whole_word_mask
from fairseq.tasks import FairseqTask, register_task


@register_task('mask_s2s')
class DenoisingTaskS2S(FairseqTask):
    """
    Denoising task for applying sequence to sequence denoising.
    """

    @staticmethod
    def add_args(parser):
        """Add task-specific arguments to the parser."""
        parser.add_argument('data', help='path to data directory')
        parser.add_argument('--tokens-per-sample', default=512, type=int,
                            help='max number of total tokens over all segments'
                                 ' per sample for dataset')
        parser.add_argument('--raw-text', default=False, action='store_true',
                            help='load raw text dataset')
        parser.add_argument(
            '--sample-break-mode', default="eos", type=str,
            help='mode for breaking sentence',
        )
        parser.add_argument(
            '--mask', default=0.3, type=float,
            help='fraction of words/subwords that will be masked',
        )
        parser.add_argument(
            '--mask-random', default=0.0, type=float,
            help='instead of using [MASK], use random token this often'
        )
        parser.add_argument(
            '--insert', default=0.0, type=float,
            help='insert this percentage of additional random tokens',
        )
        parser.add_argument(
            '--mask-length', default="word", type=str,
            choices=['subword', 'word', 'span', 'span-poisson'],
            help='mask length to choose'
        )
        parser.add_argument(
            '--replace-length', default=-1, type=int,
            help='when masking N tokens, replace with 0, 1, or N tokens (use -1 for N)'
        )
        parser.add_argument(
            '--tokens-to-keep', default=2, type=int,
            help="Don't mask first <tokens-to-keep> tokens"
        )

        # following 2 arguments are required for the GPT2 BPE encoding
        parser.add_argument(
            '--gpt2_encoder_json', default=None, type=str,
            help='GPT2 encoder path'
        )
        parser.add_argument(
            '--gpt2_vocab_bpe', default=None, type=str,
            help='GPT2 vocab path'
        )

    def __init__(self, args, dictionary):
        super().__init__(args)
        self.dictionary = dictionary
        self.seed = args.seed

        # add mask token
        self.mask_idx = self.dictionary.add_symbol('<mask>')

    @classmethod
    def setup_task(cls, args, **kwargs):
        """Setup the task.
        """
        dictionary = Dictionary.load(os.path.join(args.data, 'dict.txt'))
        print('| dictionary: {} types'.format(len(dictionary)))
        if not hasattr(args, 'shuffle_instance'):
            args.shuffle_instance = False
        return cls(args, dictionary)

    def load_dataset(self, split, epoch=0, combine=False, data_selector=None):
        """Load a given dataset split.

        Args:
            split (str): name of the split (e.g., train, valid, test)
        """

        paths = self.args.data.split(':')
        assert len(paths) > 0
        data_path = paths[epoch % len(paths)]
        split_path = os.path.join(data_path, split)

        dataset = data_utils.load_indexed_dataset(
            split_path,
            self.dictionary,
            self.args.dataset_impl,
            combine=combine,
        )
        if dataset is None:
            raise FileNotFoundError('Dataset not found: {} ({})'.format(split, split_path))

        dataset = StripTokenDataset(dataset, self.dictionary.eos())

        # create continuous blocks of tokens
        dataset = TokenBlockDataset(
                dataset,
                dataset.sizes,
                self.args.tokens_per_sample - 2,  # one less for <s> and one for </s>
                pad=self.dictionary.pad(),
                eos=self.dictionary.eos(),
                break_mode="eos",
                document_sep_len=0
        )

        # prepend beginning-of-sentence token (<s>, equiv. to [CLS] in BERT)
        dataset = PrependTokenDataset(dataset, self.source_dictionary.bos())
        dataset = AppendTokenDataset(dataset, self.source_dictionary.eos())

        mask_whole_words = get_whole_word_mask(self.args, self.source_dictionary) \
            if self.args.mask_length != 'subword' else None

        self.datasets[split] = BARTDenoisingDataset(
            dataset, dataset.sizes, self.dictionary, self.mask_idx,
            mask_whole_words, shuffle=self.args.shuffle_instance,
            seed=self.seed, args=self.args
        )
        print(
            "| Split: {0}, Loaded {1} samples of denoising_dataset".format(
                split,
                len(self.datasets[split]),
            )
        )

    def max_positions(self):
        """Return the max sentence length allowed by the task."""
        #return (self.args.max_source_positions, self.args.max_target_positions)
        return (1024, 1024)

    @property
    def source_dictionary(self):
        """Return the source :class:`~fairseq.data.Dictionary`."""
        return self.dictionary

    @property
    def target_dictionary(self):
        """Return the target :class:`~fairseq.data.Dictionary`."""
        return self.dictionary

In [ ]:
#!/usr/bin/env bash

WARMUP_UPDATES=60
LR=1e-05                # Peak LR for polynomial LR scheduler.
SRC=~/PretrainedDataAugment/src
BART_PATH=~/bart.large
CACHE=~/CACHE
PREFIXSIZE=3
MAXEPOCH=30
TASK=snips

for NUMEXAMPLES in 10;
do
    for i in {0..14};
    do
    RAWDATADIR=~/datasets/${TASK}/exp_${i}_${NUMEXAMPLES}
    DATABIN=$RAWDATADIR/jointdatabin

    splits=( train dev )
    for split in "${splits[@]}";
        do
        python $SRC/utils/bpe_encoder.py \
            --encoder-json $SRC/utils/gpt2_bpe/encoder.json \
            --vocab-bpe $SRC/utils/gpt2_bpe/vocab.bpe \
            --inputs $RAWDATADIR/${split}.tsv  \
            --outputs $RAWDATADIR/${split}_bpe.src \
            --workers 1 --keep-empty --tsv --dataset $TASK
        done

        fairseq-preprocess --user-dir=$SRC/bart_aug --only-source \
                    --task mask_s2s \
                    --trainpref $RAWDATADIR/train_bpe.src \
                    --validpref $RAWDATADIR/dev_bpe.src \
                    --destdir $DATABIN \
                    --srcdict $BART_PATH/dict.txt

        # Run data generation with different noise setting
        for mr in 40;
          do
            MRATIO=0.${mr}
            for MASKLEN in word span;
                do
                MODELDIR=$RAWDATADIR/bart_${MASKLEN}_mask_${MRATIO}_checkpoints
                mkdir $MODELDIR

                CUDA_VISIBLE_DEVICES=0 fairseq-train  $DATABIN/ \
                    --user-dir=$SRC/bart_aug \
                    --restore-file $BART_PATH/model.pt \
                    --arch bart_large \
                    --task mask_s2s \
                    --bpe gpt2 \
                    --gpt2_encoder_json $SRC/utils/gpt2_bpe/encoder.json \
                    --gpt2_vocab_bpe $SRC/utils/gpt2_bpe/vocab.bpe \
                    --layernorm-embedding \
                    --share-all-embeddings \
                    --save-dir $MODELDIR\
                    --seed $i \
                    --share-decoder-input-output-embed \
                    --reset-optimizer --reset-dataloader --reset-meters \
                    --required-batch-size-multiple 1 \
                    --max-tokens 2000 \
                    --criterion label_smoothed_cross_entropy --label-smoothing 0.1 \
                    --dropout 0.1 --attention-dropout 0.1 \
                    --weight-decay 0.01 --optimizer adam --adam-betas "(0.9, 0.98)" --adam-eps 1e-08 \
                    --clip-norm 0.0 \
                    --lr-scheduler polynomial_decay --lr $LR \
                    --warmup-updates $WARMUP_UPDATES \
                    --replace-length 1 --mask-length $MASKLEN --mask $MRATIO --fp16 --update-freq 1 \
                    --max-epoch $MAXEPOCH --no-epoch-checkpoints > $MODELDIR/bart.log

               CUDA_VISIBLE_DEVICES=0 fairseq-generate $DATABIN \
                        --user-dir=$SRC/bart_aug \
                        --task mask_s2s --tokens-to-keep $PREFIXSIZE \
                        --seed ${i} \
                        --bpe gpt2 \
                        --gpt2_encoder_json $SRC/utils/gpt2_bpe/encoder.json \
                        --gpt2_vocab_bpe $SRC/utils/gpt2_bpe/vocab.bpe \
                        --path $MODELDIR/checkpoint_best.pt \
                        --replace-length 1 --mask-length $MASKLEN --mask $MRATIO \
                        --batch-size 64 --beam 5 --lenpen 5 \
                        --no-repeat-ngram-size 2 \
                        --max-len-b 50 --prefix-size $PREFIXSIZE \
                        --gen-subset train > $MODELDIR/bart_l5_${PREFIXSIZE}.gen

                grep ^H $MODELDIR/bart_l5_${PREFIXSIZE}.gen | cut -f3 > $MODELDIR/bart_l5_gen_${PREFIXSIZE}.bpe
                rm $MODELDIR/checkpoint_last.pt
                python $SRC/utils/bpe_encoder.py \
                        --encoder-json $SRC/utils/gpt2_bpe/encoder.json \
                        --vocab-bpe $SRC/utils/gpt2_bpe/vocab.bpe \
                        --inputs $MODELDIR/bart_l5_gen_${PREFIXSIZE}.bpe \
                        --outputs $MODELDIR/bart_l5_gen_${PREFIXSIZE}.tsv --dataset $TASK \
                        --workers 1 --keep-empty --decode --tsv
            done
        done

        ########################
        ## BART Classifier
        ########################

         for mr in 40;
              do
                MRATIO=0.${mr}
                for MASKLEN in span word;
                    do
                     MODELDIR=$RAWDATADIR/bart_${MASKLEN}_mask_${MRATIO}_checkpoints

                    cat $RAWDATADIR/train.tsv $MODELDIR/bart_l5_gen_${PREFIXSIZE}.tsv > $MODELDIR/train.tsv
                    cp $RAWDATADIR/test.tsv $MODELDIR/test.tsv
                    cp $RAWDATADIR/dev.tsv $MODELDIR/dev.tsv
                    python $SRC/bert_aug/bert_classifier.py --task $TASK --data_dir $MODELDIR --seed ${i} --cache $CACHE > $RAWDATADIR/bert_bart_l5_${MASKLEN}_mask_${MRATIO}_prefix_${PREFIXSIZE}.log
                done
           done
    done
done